# Using the ORM to store several boreholes

Imports

In [18]:
from os import remove
from utils.io import boreholes_from_files, striplog_from_text
from core.core import Project
from core.orm import BoreholeOrm, PositionOrm, Base
from striplog import Lexicon, Striplog
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

In [19]:
remove('tfe_orm_db.db')

Create the db engine and the database structure

In [20]:
engine = create_engine('sqlite:///tfe_orm_db.db', echo=True)

In [21]:
Base.metadata.create_all(engine)

2021-02-01 09:08:53,561 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-02-01 09:08:53,562 INFO sqlalchemy.engine.base.Engine ()
2021-02-01 09:08:53,563 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-02-01 09:08:53,564 INFO sqlalchemy.engine.base.Engine ()
2021-02-01 09:08:53,565 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boreholes")
2021-02-01 09:08:53,566 INFO sqlalchemy.engine.base.Engine ()
2021-02-01 09:08:53,568 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Boreholes")
2021-02-01 09:08:53,569 INFO sqlalchemy.engine.base.Engine ()
2021-02-01 09:08:53,570 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Positions")
2021-02-01 09:08:53,571 INFO sqlalchemy.engine.base.Engine ()
2021-02-01 09:08:53,572 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Positions")
2021-02-01 09:08:53,574 INFO sqlalchemy.engine.base.Engine ()
2021-02-01

### Create the objects from las files

In [22]:
lexicon = Lexicon.default()

In [35]:
borehole_dict = {'F1':ROOT_DIR+'/data/test.las', 'F2':ROOT_DIR+'/data/test.csv'} # boreholes to insert into the db

In [36]:
boreholes, components= boreholes_from_files(borehole_dict, verbose=0)

part A |  F1
part A |  F2
part B |  2  |  5  |  2  |  F3
part B |  3  |  10  |  3  |  F4


Create the session

In [27]:
Session = sessionmaker(bind=engine)
session = Session()

Create the project

In [28]:
p = Project(session)
p.add_components(components)

2021-02-01 09:08:54,545 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-01 09:08:54,546 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"
2021-02-01 09:08:54,547 INFO sqlalchemy.engine.base.Engine ()
2021-02-01 09:08:54,551 INFO sqlalchemy.engine.base.Engine INSERT INTO "Components" (id, description) VALUES (?, ?)
2021-02-01 09:08:54,552 INFO sqlalchemy.engine.base.Engine ((0, 'Siltstone, grey'), (1, 'Anhydrite'), (2, 'Sandstone, vf-f, grey'), (3, 'Dolomite'), (4, 'Siltstone, red'), (5, 'Limestone'))
2021-02-01 09:08:54,555 INFO sqlalchemy.engine.base.Engine COMMIT
2021-02-01 09:08:54,700 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-01 09:08:54,701 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"
2021-02-01 09:08:54,702 INFO sqlalchemy.engine.base.Engine ()


Add boreholes into the database

In [29]:
for bh in boreholes:
    p.add_borehole(bh)

2021-02-01 09:08:54,713 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id) VALUES (?)
2021-02-01 09:08:54,714 INFO sqlalchemy.engine.base.Engine ('F1',)
2021-02-01 09:08:54,718 INFO sqlalchemy.engine.base.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-02-01 09:08:54,718 INFO sqlalchemy.engine.base.Engine ((0, 14.3, 14.3, 14.3, 0.0, 0.0), (1, 51.24117647058824, 51.24117647058824, 51.24117647058824, 0.0, 0.0), (2, 51.24117647058824, 51.24117647058824, 51.24117647058824, 0.0, 0.0), (3, 54.81764705882354, 54.81764705882354, 54.81764705882354, 0.0, 0.0), (4, 54.81764705882354, 54.81764705882354, 54.81764705882354, 0.0, 0.0), (5, 56.55882352941177, 56.55882352941177, 56.55882352941177, 0.0, 0.0), (6, 56.55882352941177, 56.55882352941177, 56.55882352941177, 0.0, 0.0), (7, 58.86470588235295, 58.86470588235295, 58.86470588235295, 0.0, 0.0)  ... displaying 10 of 50 total bound parameter sets ...  (48, 131.47647058823532, 131.4764705882

2021-02-01 09:08:54,938 INFO sqlalchemy.engine.base.Engine (14,)
2021-02-01 09:08:54,942 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 09:08:54,943 INFO sqlalchemy.engine.base.Engine (15,)
2021-02-01 09:08:54,945 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 09:08:54,947 INFO sqlalchemy.engine.base.Engine (16,)
2021-02-01 09:08:54,948 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

2021-02-01 09:08:55,038 INFO sqlalchemy.engine.base.Engine (35,)
2021-02-01 09:08:55,040 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 09:08:55,042 INFO sqlalchemy.engine.base.Engine (36,)
2021-02-01 09:08:55,044 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 09:08:55,044 INFO sqlalchemy.engine.base.Engine (37,)
2021-02-01 09:08:55,046 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

2021-02-01 09:08:55,335 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 09:08:55,336 INFO sqlalchemy.engine.base.Engine (50,)
2021-02-01 09:08:55,339 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 09:08:55,340 INFO sqlalchemy.engine.base.Engine (51,)
2021-02-01 09:08:55,342 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Position

2021-02-01 09:08:55,413 INFO sqlalchemy.engine.base.Engine (70,)
2021-02-01 09:08:55,415 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 09:08:55,416 INFO sqlalchemy.engine.base.Engine (71,)
2021-02-01 09:08:55,418 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 09:08:55,420 INFO sqlalchemy.engine.base.Engine (72,)
2021-02-01 09:08:55,423 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

...

2021-02-01 09:08:55,765 INFO sqlalchemy.engine.base.Engine (105,)
2021-02-01 09:08:55,768 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 09:08:55,769 INFO sqlalchemy.engine.base.Engine (106,)
2021-02-01 09:08:55,771 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 09:08:55,772 INFO sqlalchemy.engine.base.Engine (107,)
2021-02-01 09:08:55,776 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Pos

2021-02-01 09:08:55,968 INFO sqlalchemy.engine.base.Engine (126,)
2021-02-01 09:08:55,971 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 09:08:55,972 INFO sqlalchemy.engine.base.Engine (127,)
2021-02-01 09:08:55,974 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 09:08:55,975 INFO sqlalchemy.engine.base.Engine (128,)
2021-02-01 09:08:55,976 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Pos

2021-02-01 09:08:56,054 INFO sqlalchemy.engine.base.Engine (147,)
2021-02-01 09:08:56,055 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 09:08:56,056 INFO sqlalchemy.engine.base.Engine (148,)
2021-02-01 09:08:56,060 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 09:08:56,061 INFO sqlalchemy.engine.base.Engine (149,)
No legend given or incorrect format ! A default legend will be used
Borehole geometry created successfully !
2021-02-01 09:08:56,209 INFO sqlalc

2021-02-01 09:08:56,343 INFO sqlalchemy.engine.base.Engine (161,)
2021-02-01 09:08:56,346 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 09:08:56,347 INFO sqlalchemy.engine.base.Engine (162,)
2021-02-01 09:08:56,348 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 09:08:56,349 INFO sqlalchemy.engine.base.Engine (163,)
2021-02-01 09:08:56,352 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Pos

2021-02-01 09:08:56,409 INFO sqlalchemy.engine.base.Engine (182,)
2021-02-01 09:08:56,411 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 09:08:56,412 INFO sqlalchemy.engine.base.Engine (183,)
2021-02-01 09:08:56,414 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 09:08:56,415 INFO sqlalchemy.engine.base.Engine (184,)
2021-02-01 09:08:56,418 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Pos

In [30]:
p.boreholes[0].id

'F1'

In [31]:
p.commit()

2021-02-01 09:08:56,736 INFO sqlalchemy.engine.base.Engine COMMIT


In [32]:
p.boreholes[0].intervals[0].description

2021-02-01 09:08:56,811 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-01 09:08:56,813 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-02-01 09:08:56,814 INFO sqlalchemy.engine.base.Engine ('F1',)
2021-02-01 09:08:56,816 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-02-01 09:08:56,817 INFO sqlalchemy.engine.base.Engine ('F1',)


'Anhydrite'

In [33]:
session.close()

2021-02-01 09:08:56,894 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [34]:
p.plot3d(x3d=True)